In [1]:
import xarray as xr
import pandas as pd
from datetime import datetime
import os, glob
import numpy as np
import metpy.calc as mpcalc
import time
import pandas as pd
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
import shapely.geometry as sgeom
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from metpy.units import units
from functools import reduce
import operator
import math

In [2]:
def getDistance(latA, lonA, latB, lonB):
    ra = 6378136.49  # radius of equator: meter
    rb = 6356755  # radius of polar: meter

    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
 
    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    if radLonA == radLonB:
        x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(0.001))
    else:
        x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB))**2 / math.cos(x / 2)**2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB))**2 / math.sin(x / 2)**2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)/1000
    return distance

In [29]:
def vort_function(longitude, latitude, u, v):
    MAX=1000 # maximum iteration (corresponding eps: 1e-7)
    epsilon=1e-5 # precision
    sor_index=0.2
    N=len(longitude)
    M=len(latitude)
    chi=np.zeros((M,N)) #initialization
    Res=np.ones((M,N))*(-9999)
    dx,dy=mpcalc.lat_lon_grid_deltas(longitude, latitude)
#     print(dx.shape,dy.shape)
    divh=mpcalc.divergence(u, v,dx=dx,dy=dy)
    divh=np.asarray(divh)
    dxx=np.asarray(dx)
    dyy=np.asarray(dy)
    for k in range(1000):
        for i in range(1,M-1):
            for j in range(1,N-1):
                Res[i, j]=(chi[i+1, j]+chi[i-1, j]-2*chi[i, j])/(dxx[i, j-1]*dxx[i, j])+(chi[i, j+1]+chi[i, j-1]-2*chi[i, j])/(dyy[i-1, j]*dyy[i, j])+divh[i, j]
                chi[i, j]=chi[i, j]+(1+sor_index)*Res[i, j]/(2/(dxx[i, j-1]*dxx[i, j])+2/(dyy[i-1, j]*dyy[i, j]))
#         print(k)
        if np.max(np.max(Res))<epsilon:
            break   #Terminate the loop
    chi=chi*units.meters*units.meters/units.seconds
    grad = mpcalc.gradient(chi,deltas=(dy,dx))
    Upsi=np.asarray(-grad[1])
    Vpsi=np.asarray(-grad[0])
    return Upsi,Vpsi

In [30]:
def stream_function(longitude, latitude, u, v):
    MAX=1000 # maximum iteration (corresponding eps: 1e-7)
    epsilon=1e-5 # precision
    sor_index=0.2
    N=len(longitude)
    M=len(latitude)
    psi=np.zeros((M,N)) #initialization
    Res=np.ones((M,N))*(-9999)
#     Res=Res/units.second
    dx,dy=mpcalc.lat_lon_grid_deltas(longitude, latitude)
    curlz=mpcalc.vorticity(u, v,dx=dx,dy=dy)
#     curlz_absolute=mpcalc.absolute_vorticity(u, v,dx=dx,dy=dy,latitude=latitude)
#     curlz_relative=curlz+curlz_absolute
    curlz=np.asarray(curlz)
    dxx=np.asarray(dx)
    dyy=np.asarray(dy)
    for k in range(1000):
        for i in range(1,M-1):
            for j in range(1,N-1):
                Res[i, j]=(psi[i+1, j]+psi[i-1, j]-2*psi[i, j])/(dxx[i, j-1]*dxx[i, j])+(psi[i, j+1]+psi[i, j-1]-2*psi[i, j])/(dyy[i-1, j]*dyy[i, j])-curlz[i, j]
                psi[i, j]=psi[i, j]+(1+sor_index)*Res[i, j]/(2/(dxx[i, j-1]*dxx[i, j])+2/(dyy[i-1, j]*dyy[i, j]))
#         print(k)
        if np.max(np.max(Res))<epsilon:
            break   #Terminate the loop
    #vorticity wind
    psi=psi*units.meters*units.meters/units.seconds
    grad = mpcalc.gradient(psi,deltas=(dy,dx))
    Vpsi=np.asarray(grad[1])
    Upsi=np.asarray(-grad[0])
    return Upsi,Vpsi

In [5]:
ds_sst=[]
for ncFile in glob.glob(os.path.join("../data/era5/era5_sst","*.nc")):
    ds=xr.open_dataset(ncFile)
    ds_sst.append(ds)

In [6]:
ds_p=[]
p_files = sorted(glob.glob(os.path.join("../data/era5/era5_p","*.nc")))
for k in range(0, len(p_files), 3):
    chunk = p_files[k:k+3]
    ds_p.append(xr.open_mfdataset(chunk, combine="by_coords"))

In [7]:
ds_u=[]
u_files = sorted(glob.glob(os.path.join("../data/era5/era5_u","*.nc")))
for k in range(0, len(u_files), 3):
    chunk = u_files[k:k+3]
    ds_u.append(xr.open_mfdataset(chunk, combine="by_coords"))

In [8]:
ds_v=[]
v_files = sorted(glob.glob(os.path.join("../data/era5/era5_v","*.nc")))
for k in range(0, len(v_files), 3):
    chunk = v_files[k:k+3]
    ds_v.append(xr.open_mfdataset(chunk, combine="by_coords"))

In [9]:
data_track=pd.read_csv(r'../data/IBTrACS_fore72.txt',sep=',',header=None,names=['name','date','lat','lon','ws','p','speed','direct'])
#每个台风的索引的集合
index_i=[]
for i in range(len(data_track)):
    if data_track.name[i]=='66666':
        index_i.append(i)   

In [ ]:
with open(r'../data/fore72_IBTrACS_uv.txt', 'a') as outfile:
    for i in range(len(index_i)):
        m=index_i[i]+1
        if i ==len(index_i)-1:
            n=len(data_track)
        else:
            n=index_i[i+1]
        sel_data=[]
        for l in range(m,n):
            lat=data_track.lat[l]
            lon=data_track.lon[l]
            date_time=data_track.date[l]
            datetime=np.datetime64(date_time,'s')
            datetime_64=pd.to_datetime(datetime)
            date=datetime_64.strftime('%Y%m%d%H')
            datetime=pd.to_datetime(date,format='%Y%m%d%H')
            year=int(date[0:4])     
            U=ds_u[year-1979].sel(valid_time=datetime)
            V=ds_v[year-1979].sel(valid_time=datetime)
            u_wind=U.u
            v_wind=V.v
            sel_ds0=u_wind.sel(latitude=lat,longitude=lon,method='nearest')
            
            center_lat = float(sel_ds0.latitude)
            center_lon = float(sel_ds0.longitude)

            if (center_lat < 10 or center_lat > 50 or
                center_lon < 100 or center_lon > 170):
                continue

            lat_0 = center_lat - 10
            lat_1 = center_lat + 10
            lon_0 = center_lon - 10
            lon_1 = center_lon + 10
            a_height=[]
            lat_vals = u_wind.latitude.values
            if lat_vals[0] > lat_vals[-1]:
                lat_slice = slice(lat_1, lat_0)
            else:
                lat_slice = slice(lat_0, lat_1)
            for height in [300,500,700,850]:
                uu=u_wind.loc[height,lat_slice,lon_0:lon_1]
                vv=v_wind.loc[height,lat_slice,lon_0:lon_1]
                #去涡旋
                ud,vd=vort_function(uu.longitude.values, uu.latitude.values, uu.values, vv.values)
                ur,vr=stream_function(uu.longitude, uu.latitude, uu, vv)
                ug=uu.values-ud-ur
                vg=vv.values-vd-vr
                u_nparray=np.array(ug).flatten()
                v_nparray=np.array(vg).flatten()
                a=np.hstack((u_nparray,v_nparray))
                a_height.append(list(a))
            aa=reduce(operator.add, a_height)
            sel_data.append(aa)
        outfile.write('666666,\n')
        np.savetxt(outfile, sel_data, fmt = '%.3f', delimiter = ',')
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11


In [ ]:
with open(r'G:\typhoon-ocean\last_test\fore72_IBTrACS_sst.txt', 'w') as outfile:
    for i in range(len(index_i)):
        m=index_i[i]+1
        if i ==len(index_i)-1:
            n=len(data_track)
        else:
            n=index_i[i+1]
        sel_data=[]
        for l in range(m,n):
            lat=data_track.lat[l]
            lon=data_track.lon[l]
            date_time=data_track.date[l]
            datetime=np.datetime64(date_time,'s')
            datetime_64=pd.to_datetime(datetime)
            date=datetime_64.strftime('%Y%m%d%H')
            datetime=pd.to_datetime(date,format='%Y%m%d%H')
            year=int(date[0:4])
            if 1979<=year<=1983:
                dataset=ds_sst[0].sel(valid_time=datetime)
            elif 1984<=year<=1988:
                dataset=ds_sst[1].sel(valid_time=datetime)
            elif 1989<=year<=1993:
                dataset=ds_sst[2].sel(valid_time=datetime)
            elif 1994<=year<=1998:
                dataset=ds_sst[3].sel(valid_time=datetime)  
            elif 2019<=year<=2021:
                dataset=ds_sst[4].sel(valid_time=datetime)    
            elif 2014<=year<=2018:
                dataset=ds_sst[5].sel(valid_time=datetime)
            elif 2009<=year<=2013:
                dataset=ds_sst[6].sel(valid_time=datetime)   
            elif 2004<=year<=2008:
                dataset=ds_sst[7].sel(valid_time=datetime) 
            elif 1999<=year<=2003:
                dataset=ds_sst[8].sel(valid_time=datetime)  
            sst=dataset.sst
            sel_ds0=sst.sel(latitude=lat,longitude=lon,method='nearest')

            center_lat = float(sel_ds0.latitude)
            center_lon = float(sel_ds0.longitude)

            if (center_lat < 10 or center_lat > 50 or
                center_lon < 100 or center_lon > 170):
                continue

            lat_0=sel_ds0.latitude-10
            lat_1=sel_ds0.latitude+10
            lon_0=sel_ds0.longitude-10
            lon_1=sel_ds0.longitude+10

            lat_vals = sst.latitude.values
            if lat_vals[0] > lat_vals[-1]:
                lat_slice = slice(lat_1, lat_0)
            else:
                lat_slice = slice(lat_0, lat_1)

            sst_a=sst.loc[lat_0:lat_1,lon_0:lon_1]
            sst_a=np.array(sst_a).flatten()
            sel_data.append(sst_a)
        outfile.write('666666,\n')
        np.savetxt(outfile, sel_data, fmt = '%.3f', delimiter = ',')
        print(i)
    #sel_essemble.append(sel_data)

In [ ]:
with open(r'G:\typhoon-ocean\last_test\fore72_IBTrACS_p.txt', 'a') as outfile:
    for i in range(len(index_i)):
        m=index_i[i]+1
        if i ==len(index_i)-1:
            n=len(data_track)
        else:
            n=index_i[i+1]
        sel_data=[]
        for l in range(m,n):
            lat=data_track.lat[l]
            lon=data_track.lon[l]
            date_time=data_track.date[l]
            datetime=np.datetime64(date_time,'s')
            datetime_64=pd.to_datetime(datetime)
            date=datetime_64.strftime('%Y%m%d%H')
            datetime=pd.to_datetime(date,format='%Y%m%d%H')
            year=int(date[0:4])     
            p=ds_p[year-1979].sel(time=datetime)
            hgt=p.z[:,::-2,::2]
            sel_ds0=hgt.sel(latitude=lat,longitude=lon,method='nearest')
            lat_0=sel_ds0.latitude-10
            lat_1=sel_ds0.latitude+35
            lon_0=sel_ds0.longitude-40
            lon_1=sel_ds0.longitude+40
            if lat_1>=90:
                lat_1=89
                lat_0=44
            p_500=hgt.loc[500,lat_0:lat_1,lon_0:lon_1]
            ssp_ds=dataset.sp[::-4,::4]
            ssp=ssp_ds.loc[lat_0:lat_1,lon_0:lon_1]
            p_500=np.array(p_500)
            hgt_nparray=p_500.flatten()
            sel_data.append(hgt_nparray)
        outfile.write('666666,\n')
        np.savetxt(outfile, sel_data, fmt = '%.3f', delimiter = ',')
        print(i)